In [1]:
#!pip install py3Dmol
# For pdb visualization
import py3Dmol

In [2]:

import sys
sys.path.append('../')
#from pyfaidx import Fasta
import numpy as np
from src.ESM_utils import ESMEmbed
from src.data_processing import PDDataProcessor
from src.prediction_utils import EsmCEsmPDModel
from src.misc import *

2023-10-27 13:25:00.043563: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-27 13:25:00.045393: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 13:25:00.086783: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 13:25:00.087435: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-27 13:25:00.745555: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

## ESM-C(ESM) Model

Load sequeuence and set model location

In [4]:
embedding_type = 'ESM' # ESM or ProteinBERT
model_loc = '../models/ESM_C_ESM/saved_model'
id, seq = read_FASTA('../data/AF2/demo/3rae.C/3rae.C.fa')
seq = seq[:128] # Get the first 128 Amino acid residues.
print(f'Protein sequence: {seq}')

Protein sequence: GKLTPAQSKNPAKNELYLVEGDSAGGSAKQGRDRKFQAILPLGKVINTAKAKMADILKNEEINTMITIGAGVGADFSIEDANYDKIIIMTDADTDGAHIQTLLLTFFYRYMPLVEAGHVYIALPPLYK


In [5]:
# Generate Embedding
ESM_seq_rpres = ESMEmbed(representation_layer=33)
X, C = ESM_seq_rpres.generate_embeddig(seq)
# Process Embedding for to make it ready for prediction 
ESM_DP = PDDataProcessor(embedding_type=embedding_type, seq_out_len=128,  scale_X = False, scale_A=False, thr_A=None, auxiliary_data='../data/minmax')
X_model, C_model, Seq_model = ESM_DP.process_data(X, C, seq, 0)
# Load ESM-C(ESM) model
ESM_PD_Predictor = EsmCEsmPDModel(model_loc)
# Predict Protein-DNA interactions 
prediction = ESM_PD_Predictor.predict(X_model, C_model)
#print(prediction)
# Load sequence labels
_, seq_labels = read_FASTA('../data/AF2/demo/3rae.C/3rae.C.labels')
seq_labels = seq_labels[:128]
seq_labels = [int(l) for l in seq_labels]
seq_labels = np.array(seq_labels)
binding_AAs = np.where(seq_labels==1)[0]
predicted_binding_AAs = np.where(prediction>=0.5)[0]

In [6]:
# characterize the prediction
from sklearn.metrics import precision_recall_fscore_support, auc, matthews_corrcoef, confusion_matrix, accuracy_score,roc_curve
import numpy as np

In [8]:
y_pred_ = np.round(prediction) 
res = precision_recall_fscore_support(seq_labels, y_pred_, average='binary', labels=[0, 1])
fpr, tpr, thresholds = roc_curve(seq_labels, prediction, pos_label=1)
auc_val = auc(fpr, tpr)
mcc = matthews_corrcoef(seq_labels, y_pred_)
acc = accuracy_score(seq_labels, y_pred_)
tn, fp, fn, tp = confusion_matrix(seq_labels, y_pred_).ravel()
specificity = tn / (tn+fp)

In [14]:
print(f'Precision: {np.round(res[0], 2)}, Recall: {np.round(res[1], 2)}, F1: {np.round(res[2], 2)}, AUC: {np.round(auc_val, 2)}, Accuracy: {np.round(acc, 2)}, Specificity: {np.round(specificity, 2)}')

Precision: 0.45, Recall: 0.77, F1: 0.57, AUC: 0.92, Accuracy: 0.88, Specificity: 0.9


In [4]:
# Code color for binding residues
# Blue: True Positive
TP_AA = [AA_ind+1 for AA_ind in list(predicted_binding_AAs) if AA_ind in list(binding_AAs)]
print(f'True positive Binding Residues {TP_AA}')
# Red: False Positive
FP_AA = [AA_ind+1 for AA_ind in list(predicted_binding_AAs) if AA_ind not in list(binding_AAs)]
print(f'False positive Binding Residues {FP_AA}')
# Orange: False Negative
FN_AA = [AA_ind+1 for AA_ind in list(binding_AAs) if AA_ind not in list(predicted_binding_AAs)]
print(f'False Negative Binding Residues {FN_AA}')
# All other True negative are green

True positive Binding Residues [20, 21, 22, 23, 43, 44, 47, 50, 95, 98]
False positive Binding Residues [24, 26, 29, 45, 91, 92, 93, 94, 96, 99, 127, 128]
False Negative Binding Residues [46, 59, 102]


Visualize pdb along with predictions

In [8]:
#First we assign the py3Dmol.view as view
view=py3Dmol.view()
#The following lines are used to add the addModel class
view.addModel(open('../data/AF2/demo/3rae.C/3rae.C.pdb', 'r').read(),'pdb')
#Here we set the background color as white
view.setBackgroundColor('white')
view.setStyle({'chain':'A'},{"cartoon": {'color': 'green'}})
view.addStyle({'chain':'A','resi':[20, 21, 22, 23, 43, 44, 47, 50, 95, 98]},{'cartoon':{'color':'blue'}})
view.addStyle({'chain':'A','resi':[24, 26, 29, 45, 91, 92, 93, 94, 96, 99, 127, 128]},{'cartoon':{'color':'orange'}})
view.addStyle({'chain':'A','resi':[46, 59, 102]},{'cartoon':{'color':'red'}})
#Zooming into all visualized structures 
view.zoomTo()
view.show()
#And we finally visualize the structures using the command below


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## AF2-C(AF2) Model

Wile ESEM-C(ESM) Model does not require running embedding generation, the process takes seconds (~1s based on hardware configuration). For AF2, it generating embedding, requires hours of running in order to generate the embeddings. We already ran AF2 for 3rae.C and stored the results locally. 


In [15]:
# Get Embeddings and distance maps from pdb file. 
from src.alphafold2_utils import AlphaFold2Embed
loc = '../data/AF2/demo/3rae.C/3rae.C.pdb'
AF2_seq_rpres = AlphaFold2Embed(caldis_CA_loc='../src/caldis_CA')
X, C, seq = AF2_seq_rpres.generate_embeddig(loc)
#print(X)
#print(C)

Process data before prediction 

In [16]:
from src.data_processing import PDDataProcessor
embedding_type = 'AF2' # ESM or ProteinBERT
ind = 0 # index of starting location. 
AF2_DP = PDDataProcessor(embedding_type=embedding_type, seq_out_len=128,  scale_X = True, scale_A=True, thr_A=None,auxiliary_data='../data/minmax/')
X_model, C_model, Seq_model = AF2_DP.process_data(X, C, seq, ind)
#print(X_model)
#print(C_model)
print(f'Protein sequence: {Seq_model}')

Protein sequence: GKLTPAQSKNPAKNELYLVEGDSAGGSAKQGRDRKFQAILPLGKVINTAKAKMADILKNEEINTMITIGAGVGADFSIEDANYDKIIIMTDADTDGAHIQTLLLTFFYRYMPLVEAGHVYIALPPLYK


In [17]:
from src.prediction_utils import AF2AF2PDModel
# Load AF2-C(AF2) model
model_loc = '../models/AF2_C_AF2/saved_model'
AF2_PD_Predictor = AF2AF2PDModel(model_loc)
# Predict Protein-DNA interactions using AF2-C(AF2) model
prediction = AF2_PD_Predictor.predict(X_model, C_model)
# Load sequence labels
_, seq_labels = read_FASTA('../data/AF2/demo/3rae.C/3rae.C.labels')
seq_labels = seq_labels[:128]
seq_labels = [int(l) for l in seq_labels]
seq_labels = np.array(seq_labels)
binding_AAs = np.where(seq_labels==1)[0]
predicted_binding_AAs = np.where(prediction>=0.5)[0]

In [18]:
# Code color for binding residues
# Blue: True Positive
TP_AA = [AA_ind+1 for AA_ind in list(predicted_binding_AAs) if AA_ind in list(binding_AAs)]
print(f'True positive Binding Residues {TP_AA}')
# Red: False Positive
FP_AA = [AA_ind+1 for AA_ind in list(predicted_binding_AAs) if AA_ind not in list(binding_AAs)]
print(f'False positive Binding Residues {FP_AA}')
# Orange: False Negative
FN_AA = [AA_ind+1 for AA_ind in list(binding_AAs) if AA_ind not in list(predicted_binding_AAs)]
print(f'False Negative Binding Residues {FN_AA}')
# All other True negative are green

True positive Binding Residues [20, 21, 22, 23, 43, 44, 47, 59, 95]
False positive Binding Residues [2, 3, 4, 7, 8, 9, 24, 26, 27, 29, 30, 31, 32, 34, 35, 60, 63, 91, 93, 94, 99, 127]
False Negative Binding Residues [46, 50, 98, 102]


In [19]:
#First we assign the py3Dmol.view as view
import py3Dmol
view=py3Dmol.view()
#The following lines are used to add the addModel class
view.addModel(open('../data/3u58_A/3u58_A.pdb', 'r').read(),'pdb')
#Zooming into all visualized structures 
#Here we set the background color as white
view.setBackgroundColor('white')
view.setStyle({'chain':'A'},{"cartoon": {'color': 'green'}})
view.addStyle({'chain':'A','resi':[20, 21, 22, 23, 43, 44, 47, 59, 95]},{'cartoon':{'color':'blue'}})
view.addStyle({'chain':'A','resi':[2, 3, 4, 7, 8, 9, 24, 26, 27, 29, 30, 31, 32, 34, 35, 60, 63, 91, 93, 94, 99, 127]},{'cartoon':{'color':'orange'}})
view.addStyle({'chain':'A','resi':[46, 50, 98, 102]},{'cartoon':{'color':'red'}})
view.zoomTo()
view.show()
#And we finally visualize the structures using the command below


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [20]:
# Calculate the prediction performance
y_pred_ = np.round(prediction) 
res = precision_recall_fscore_support(seq_labels, y_pred_, average='binary', labels=[0, 1])
fpr, tpr, thresholds = roc_curve(seq_labels, prediction, pos_label=1)
auc_val = auc(fpr, tpr)
mcc = matthews_corrcoef(seq_labels, y_pred_)
acc = accuracy_score(seq_labels, y_pred_)
tn, fp, fn, tp = confusion_matrix(seq_labels, y_pred_).ravel()
specificity = tn / (tn+fp)

In [21]:
# Print the prediction performance
print(f'Precision: {np.round(res[0], 2)}, Recall: {np.round(res[1], 2)}, F1: {np.round(res[2], 2)}, AUC: {np.round(auc_val, 2)}, Accuracy: {np.round(acc, 2)}, Specificity: {np.round(specificity, 2)}')

Precision: 0.29, Recall: 0.69, F1: 0.41, AUC: 0.83, Accuracy: 0.8, Specificity: 0.81
